In [ ]:
%pylab inline
import qiskit as qt
from qiskit.aqua.components.iqfts import Standard as IQFT
simulator = qt.Aer.get_backend('statevector_simulator')
from scipy.sparse import coo_matrix

In [ ]:
#initialization test
dim = 2
control_register = qt.QuantumRegister(2 * dim, 'control')
target_register = qt.QuantumRegister(dim, 'target')

state = array([rand() for i in range(2 ** (3 * dim))])
state = state / sqrt(np.sum([i**2 for i in state]))

circuit = qt.QuantumCircuit(control_register,target_register,name='mine')
circuit.initialize(array(state),circuit.qubits)
returned_state = qt.execute(circuit,simulator,shots=1).result()
print(returned_state.data())

In [ ]:
constructor = IQFT(control_register)
IQFT_circuit = qt.QuantumCircuit(control_register,target_register)
constructor.construct_circuit(mode='circuit', circuit=IQFT_circuit, qubits=control_register)
IQFT_circuit.draw()
circuit = IQFT_circuit#circuit.combine(IQFT_circuit)

circuit.draw('mpl')

In [ ]:
def nBitQFT(q,c,n):
    circuit = qt.QuantumCircuit(q,c)
    #
    # We start with the most significant bit
    #
    for k in range(n):
        j = n - k
        # Add the Hadamard to qubit j-1
        circuit.h(q[j-1])
        #
        # there is one conditional rotation for
        # each qubit with lower significance
        for i in reversed(range(j-1)):
            circuit.cu1(2*np.pi/2**(j-i),q[i], q[j-1])
    #
    # Finally we need to swap qubits
    #
    for i in range(n//2):
        circuit.swap(q[i], q[n-i-1])
    return circuit

In [ ]:
n=3
q = qt.QuantumRegister(n,"x")
c = qt.ClassicalRegister(n,"c")
circuit = nBitQFT(q,c,n)
circuit.inverse().draw('mpl')

In [ ]:
#initialization order test: now it's all fine
L = 1

control_register = qt.QuantumRegister(2 * L, 'control')
target_register = qt.QuantumRegister(L, 'target')
#classical_register = qt.ClassicalRegister(2)

circuit = qt.QuantumCircuit(control_register, target_register)

# 80 percent 001, 20 of 010
circuit.initialize([0,2/sqrt(5),1/sqrt(5),0,0,0,0,0], [i for i in reversed(circuit.qubits)])


job = qt.execute(circuit, simulator, shots=1)
result = job.result()
#print(result)
print(result.results[0])
print(result.get_statevector('circuit22'))
# Grab results from the job
#result = job.result()

# Returns counts
#counts = result.get_counts()
#print(result.get_statevector('id')) 
del circuit

In [ ]:
from qiskit.aqua.circuits import FourierTransformCircuits as IQFT

L = 1
circuit = qt.QuantumCircuit(control_register,target_register)
circuit.initialize(np.array([0,0,1/2,0,1/2,0,1/2,1/2]), [i for i in reversed(circuit.qubits)])

IQFT.construct_circuit(circuit=circuit, qubits=circuit.qubits[:2 * L], inverse=True)
circuit.draw('mpl')

In [ ]:
def decimal_to_binary(number: int, length: int) -> str:
    """
        Returns binary representation of number as string of fixed length
        ----------
        Example:
        decimal_to_binary(2,5) will yield "00010"

    :param number: decimal number to be converted in binary form
    :param length:  minimum length of returned string
    :return: binary representation of decimal as string of determined length
    """

    return np.binary_repr(number, length)


def to_decimal(string_list: list) -> int:
    """
        converts a list of char representing a binary number and returns its decimal representation as integer
     :param    string_list: list of binary chars representing an integer number
     :return:  decimal representation of string_list
    """

    string = "".join(string_list)
    return int(string, 2)


def invert_qubits_state(state: coo_matrix, length: int) -> coo_matrix:
    new_idx = range(2 ** length)
    result = np.array(list(map(lambda i: to_decimal(decimal_to_binary(i, length)[::-1]), new_idx)), dtype=np.int64)
    return state.toarray().reshape((2 ** length),)[result]

In [ ]:
from qiskit.aqua.circuits import FourierTransformCircuits as QFT
L = 2
current_state = coo_matrix(np.ones(2 ** (3 * L)) / sqrt(2 ** (3*L)))
circuit = qt.QuantumCircuit(3 * L)
circuit.initialize(invert_qubits_state(current_state, 3 * L), circuit.qubits)
circuit = QFT.construct_circuit(circuit=circuit, qubits=circuit.qubits[L:3*L], inverse=True, do_swaps=True)
circuit.draw('mpl')

In [1]:
import qiskit as qt
import numpy as np

simulator = qt.Aer.get_backend('statevector_simulator')

circ = qt.QuantumCircuit(3)
#state = 001
state = np.array([0,1,0,0,0,0,0,0])
circ.initialize(state,circ.qubits)
circ.h(0)
res = qt.execute(circ, simulator).result().get_statevector(circ)
print(res)

[ 0.70710678-8.65956056e-17j -0.70710678+8.65956056e-17j
  0.        +0.00000000e+00j  0.        +0.00000000e+00j
  0.        +0.00000000e+00j  0.        +0.00000000e+00j
  0.        +0.00000000e+00j  0.        +0.00000000e+00j]
